# 품질검사성적서 등록 목록

In [1]:
from common import commonFunc as cf
import pandas as pd
pd.set_option('display.max_columns', None)
metadata = pd.read_excel("../input/datalake_meta22.xlsx")

SITENAME = "건설사업정보시스템"
DATANAME= "품질검사성적서 등록 목록"
APIKEY = "1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF"

targetData = metadata.loc[metadata.자료명==DATANAME]
preSetFolder = targetData["저장폴더"].values[0]

preSetFolder

'../output/건설사업정보시스템/품질검사성적서 등록 목록/selectIoPmQtscList.csv'

In [2]:
targetData = metadata.loc[metadata.자료명==DATANAME]

##### 파라미터 설정 #####
URL = targetData["URL"].values[0]
SERVICENAME = targetData["서비스키"].values[0]
SERVICENAME = SERVICENAME.split(".")[0]
REQPARAM = targetData["요청변수"].values[0]
REQPARAM = REQPARAM.split(",")
PRIMARYKEY = targetData["기본키"].values[0]

In [3]:
JSONKEY = "items"
DUMMY = 0

In [4]:
#### 페이지번호 설정 ###
PAGEYN=1
if REQPARAM.count("pageyn") == 0:
    PAGEYN = 0
else:
    PAGEYN = 1

In [5]:
#임시 
imsiDf = pd.read_csv(\
         "../output/건설사업정보시스템/건설자재 품질검사 등록정보/selectIoPmQtlTsitStsList.csv",low_memory=False, encoding="ms949")

In [6]:
dfCombi = imsiDf.loc[:,["cstrnm"]].drop_duplicates()
dfCombi.reset_index(inplace=True, drop=True)
dfCombiLen = dfCombi.shape[0]
dfCombiLen

74882

In [7]:
resultDfMerged = pd.DataFrame()

In [8]:
import datetime

currentDateTime = datetime.datetime.now()
date = currentDateTime.date()
endYr = date.strftime("%Y")
endYr

'2022'

In [9]:
# 기준년도 산정
# 2001년 부터 데이터 존재
startYr = "2001"
dateRange = list(map(int,pd.period_range(startYr,endYr,freq='Y').strftime('%Y').tolist()))
# dateRange

## 품질검사성적서 등록정보 전용 스크랩 함수

In [10]:
import requests
from lxml import html
from bs4 import BeautifulSoup 
import pandas as pd
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
import os

STDENCODING="utf-8"
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
### 파마리터정의: 
###   - inurl: 메타정보의 "URL"컬럼값 (예: https://www.calspia.go.kr/io/openapi/cm/selectIoCmConstructionList.do )
###   - inSiteName: 메타정보의 "자료대상" (예: 건설사업정보시스템)
###   - inDataName: 메타정보의 "자료명" (예: 공사정보 목록)
###   - inServiceName: 메타정보의 "기본키" (예: serviceKey + Format)
###   - inParam: 메타정보의 "파라미터 정보" (예: 페이지 파라미터 존재 시 1 값")
###   - inPageYn: 메타정보의 "페이지 정보" (예: 페이지 파라미터 존재 시 1 값")
### 함수정의: 사이트 메타정보를 받아 데이터를 수집 후 수집결과를 반환하는 함수
def myscrapy(inUrl, inSiteName, inDataName, inServiceName, inParam, inPageYn, jsonkey="items", dummy=0, inType="jsonabnormal",x=-1):
    try:
        emptyPd = pd.DataFrame()
        i=1
        while True:
            print("{} page scraping start".format(i))
            
            if(inPageYn==1):
                inParam["pageNo"] = i
            queryParams = '?' + urlencode(inParam)
            requests.packages.urllib3.disable_warnings(requests.packages.urllib3.exceptions.InsecureRequestWarning)
            response = requests.get(inUrl+queryParams,verify=False)
            response.encoding=STDENCODING
            rowData = pd.DataFrame()
            if(inType=="jsonabnormal"):
                # 비정상 데이터는 response 섹션이 없음
                if(response.json().get('response') == None):
                    jsondata = response.json()["header"]["resultMsg"]
                    if( jsondata != "NORMAL_SERVICE"):
                        print("SERVER ERROR ",jsondata)
                        break
                    
#               여기서부터는 데이터는 있는데, totalcount>0 경우는 items 정보에 세부정보가 더 있음
               
                jsondata = response.json()["response"]["body"][jsonkey]
                if( jsondata == []):
                    print("{} page is empty".format(i))
                    break
                if( jsonkey == "detail1"):
                    jsondata["index"]=[0]
                rowData = pd.DataFrame(jsondata)
                emptyPd = emptyPd.append(rowData)

                print(">totalCount "+str(response.json()["response"]["body"]["totalCount"]))
                if(0 < int(response.json()["response"]["body"]["totalCount"])):
                    global g_totalCount
                    if g_totalCount[x] == 0:
                        g_totalCount[x] = int(response.json()["response"]["body"]["totalCount"])
                    
            else:
                print("Error")          
     
            if(inPageYn == 0):
                print("{} no pageNo".format(inPageYn))
                break
            i = i+1

        emptyPd.columns = emptyPd.columns.str.lower()
        emptyPd.shape
#         print("dataframe{}, param:{} rows: {} completed".format(inDataName,inParam, emptyPd.shape[1] )     )
        return emptyPd       
    except Exception as e:
            print(e)     

In [11]:
numOfThread = 10

resultDfMerged = list(pd.DataFrame() for x in range(numOfThread))

g_totalCount = list(range(numOfThread))

In [12]:
def threadReq(startIdx, endIdx,x):
    numOfRows = 1000 # 서버에서 최대 응답 가능한 row임
    resultDf = pd.DataFrame()
    for i in range(startIdx,endIdx):
        searchCstrNm = dfCombi.loc[i].cstrnm
        for j  in dateRange:
            pageNo = 1
            print(searchCstrNm," ",j)
            global g_totalCount
            g_totalCount[x] = 0
            while True:
                BASEPARAM={"serviceKey":APIKEY,"searchCstrNm": searchCstrNm, "searchRpcdIsBgDt": str(j)+"0101","searchRpcdIsEdDt": str(j)+"1231", "sortField": "tstBgDt","pageNo": pageNo, 'numOfRows':numOfRows, "type":"json"}
                print("BASEPARAM ",BASEPARAM)
                resultDf = myscrapy(URL,SITENAME,DATANAME,SERVICENAME,BASEPARAM,PAGEYN, JSONKEY, DUMMY, "jsonabnormal",x)
                if resultDf.empty: # 정상 데이터가 없는 경우
                    print("No Data")
                    break
                else:
                    resultDfMerged[x] = resultDfMerged[x].append(resultDf) 
                    g_totalCount[x] -= numOfRows
                    print("x : ",x," g_totalCount : ", g_totalCount[x], " numOfRows : ", numOfRows)
                    if(g_totalCount[x] <= 0):
                        break
                    pageNo += 1

In [13]:
import threading

resultDfLastMerged = pd.DataFrame()
section = list(range(0, dfCombiLen, int(dfCombiLen/numOfThread))) # we have "numåOfThread +1" sections
section

[0, 7488, 14976, 22464, 29952, 37440, 44928, 52416, 59904, 67392, 74880]

In [ ]:
# dfCombiLen = 6 # for test
if dfCombiLen < numOfThread:
    print("데이터의 갯수가 적어서 쓰레드를 시작할 수 없습니다.")
else:
    x_ls =list(range(numOfThread))
    thread_list = []
    results = []
    idx = 0
    for x in x_ls:
        startIdx = section[x]
        endIdx =  section[x+1] -1 if (x+1 <= numOfThread -1) else dfCombiLen - 1
        print("start ",startIdx, " end ",endIdx)
        mythread = threading.Thread(target=threadReq, args=(startIdx, endIdx+1,idx))
        idx += 1
        thread_list.append(mythread)
    for i in thread_list:
        i.start()
    for i in thread_list:
        i.join()
    for i in range(0,numOfThread):
        resultDfLastMerged = resultDfLastMerged.append(resultDfMerged[i]) 



start  0  end  7487
start  7488  end  14975
start  14976  end  22463
start  22464  end  29951
start  29952  end  37439
start  37440  end  44927
start  44928  end  52415
start  52416  end  59903
start  59904  end  67391
start  67392  end  74881
현대건설(주)   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '현대건설(주)', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
남광토건(주)외2개사   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '남광토건(주)외2개사', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
(유)시온종합건설   2001
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(유)시온종합건설', 'searchRpcdIsBgDt': '20010101', 'searchRpcdIsEdDt': '20011231', 'sortField': 'tstBgDt', 'pageNo': 1, 'nu

1 page is empty
No Data
중흥종합건설(주),(주)시티건설, (주)시티이엔씨   2003
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '중흥종합건설(주),(주)시티건설, (주)시티이엔씨', 'searchRpcdIsBgDt': '20030101', 'searchRpcdIsEdDt': '20031231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
남광토건(주)외2개사   2004
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '남광토건(주)외2개사', 'searchRpcdIsBgDt': '20040101', 'searchRpcdIsEdDt': '20041231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
대승토건㈜   2004
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '대승토건㈜', 'searchRpcdIsBgDt': '20040101', 'searchRpcdIsEdDt': '20041231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
(주)흥진토건   2004
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A

1 page is empty
No Data
㈜흥건, ㈜영도종합건설		   2006
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '㈜흥건, ㈜영도종합건설\t\t', 'searchRpcdIsBgDt': '20060101', 'searchRpcdIsEdDt': '20061231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty1 page is empty
No Data
(주)황토종합건설 외 1개사   2006
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)황토종합건설 외 1개사', 'searchRpcdIsBgDt': '20060101', 'searchRpcdIsEdDt': '20061231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start

No Data
(주)흥진토건   2007
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)흥진토건', 'searchRpcdIsBgDt': '20070101', 'searchRpcdIsEdDt': '20071231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
대승토건㈜   2007
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476

1 page is empty
No Data
현대건설(주)   2009
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '현대건설(주)', 'searchRpcdIsBgDt': '20090101', 'searchRpcdIsEdDt': '20091231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
㈜흥건, ㈜영도종합건설		   2009
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '㈜흥건, ㈜영도종합건설\t\t', 'searchRpcdIsBgDt': '20090101', 'searchRpcdIsEdDt': '20091231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
(주)흥진토건   2010
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)흥진토건', 'searchRpcdIsBgDt': '20100101', 'searchRpcdIsEdDt': '20101231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
중흥종합건설(주),(주)시티건설, (주)시티이엔씨   2008
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5D

>totalCount 27
0 no pageNo
x :  1  g_totalCount :  -973  numOfRows :  1000
남광토건(주)외2개사   2012
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '남광토건(주)외2개사', 'searchRpcdIsBgDt': '20120101', 'searchRpcdIsEdDt': '20121231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
(주)황토종합건설 외 1개사   2011
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)황토종합건설 외 1개사', 'searchRpcdIsBgDt': '20110101', 'searchRpcdIsEdDt': '20111231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 1
0 no pageNo
x :  3  g_totalCount :  -999  numOfRows :  1000
서울9호선운영(주)   2012
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '서울9호선운영(주)', 'searchRpcdIsBgDt': '20120101', 'searchRpcdIsEdDt': '20121231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start

...

1 page is empty
No Data
미지개발(주)   2006
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '미지개발(주)', 'searchRpcdIsBgDt': '20060101', 'searchRpcdIsEdDt': '20061231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 3144
0 no pageNo
x :  0  g_totalCount :  2144  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '현대건설(주)', 'searchRpcdIsBgDt': '20130101', 'searchRpcdIsEdDt': '20131231', 'sortField': 'tstBgDt', 'pageNo': 2, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
금호산업 외 6개사   2006
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '금호산업 외 6개사', 'searchRpcdIsBgDt': '20060101', 'searchRpcdIsEdDt': '20061231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
동부건널(주)   2008
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A

1 page is empty
No Data
미지개발(주)   2009
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '미지개발(주)', 'searchRpcdIsBgDt': '20090101', 'searchRpcdIsEdDt': '20091231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
(주)한일건설　　　　   2009
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)한일건설\u3000\u3000\u3000\u3000', 'searchRpcdIsBgDt': '20090101', 'searchRpcdIsEdDt': '20091231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
금호산업 외 6개사   2009
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '금호산업 외 6개사', 'searchRpcdIsBgDt': '20090101', 'searchRpcdIsEdDt': '20091231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
동부건널(주)   2011
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2

1 page is empty
No Data
(주)서호씨앤디   2013
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)서호씨앤디', 'searchRpcdIsBgDt': '20130101', 'searchRpcdIsEdDt': '20131231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
에코이앤씨(주)   2014
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '에코이앤씨(주)', 'searchRpcdIsBgDt': '20140101', 'searchRpcdIsEdDt': '20141231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 4
0 no pageNo
x :  3  g_totalCount :  -996  numOfRows :  1000
미지개발(주)   2012
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '미지개발(주)', 'searchRpcdIsBgDt': '20120101', 'searchRpcdIsEdDt': '20121231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
(주)한일건설　　　　   2012
BASEPARAM  {'serviceKey': '1EFCBE

1 page is empty
No Data
(주)서호씨앤디   2016
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '(주)서호씨앤디', 'searchRpcdIsBgDt': '20160101', 'searchRpcdIsEdDt': '20161231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
한국남부발전(주)   2013
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '한국남부발전(주)', 'searchRpcdIsBgDt': '20130101', 'searchRpcdIsEdDt': '20131231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
1 page is empty
No Data
미지개발(주)   2015
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '미지개발(주)', 'searchRpcdIsBgDt': '20150101', 'searchRpcdIsEdDt': '20151231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start
>totalCount 3180
0 no pageNo
x :  0  g_totalCount :  2180  numOfRows :  1000
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A

1 page is empty
No Data
에코이앤씨(주)   2020
BASEPARAM  {'serviceKey': '1EFCBE0D-3203-4485-A7BA-5F5DB2B476BF', 'searchCstrNm': '에코이앤씨(주)', 'searchRpcdIsBgDt': '20200101', 'searchRpcdIsEdDt': '20201231', 'sortField': 'tstBgDt', 'pageNo': 1, 'numOfRows': 1000, 'type': 'json'}
1 page scraping start


In [ ]:
resultDfLastMerged

In [ ]:
cf.savedata(resultDfLastMerged, SITENAME,DATANAME,SERVICENAME)

In [ ]:
resultDfLastMerged.shape